# Getting Started with AmpliGraph

In this tutorial we will demonstrate how to use the AmpliGraph library. 

Things we will cover:

1. Exploration of a graph dataset
2. Splitting graph datasets into train and test sets
3. Training and Evaluation a Model
4. Saving and restoring a model
5. Predicting New Links
6. Visualizing embeddings using Tensorboard



## Requirements

A Python environment with the AmpliGraph library installed. Please follow [the install guide](http://docs.ampligraph.org/en/latest/install.html).



Some sanity check:

In [1]:
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

'2.0-dev'


## 1. Dataset Exploration

First things first! Lets import the required libraries and retrieve some data.

In this tutorial we're going to use the [**Game of Thrones knowledge Graph**](https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv). Please note: this isn't the *greatest* dataset for demonstrating the power of knowledge graph embeddings, but is small, intuitive and should be familiar to most users. 

We downloaded the [neo4j graph published here](https://github.com/neo4j-examples/game-of-thrones). Such dataset has been generated using [these APIs](https://anapioficeandfire.com/)  which expose in a machine-readable fashion the content of open free sources such as A [Wiki of Ice and Fire](http://awoiaf.westeros.org/). We discarded all properties and saved all the directed, labeled relations in a plaintext file. Each relation (i.e. a triple) is in the form: 

    <subject, predicate, object>

The schema of the graph looks like this (image from [neo4j-examples/game-of-thrones](https://github.com/neo4j-examples/game-of-thrones)):

![](img/got-graphql-schema.jpg)

Run the following cell to pull down the dataset and load it in memory with AmpliGraph [`load_from_csv()`](http://docs.ampligraph.org/en/1.0.3/generated/ampligraph.datasets.load_from_csv.html#ampligraph.datasets.load_from_csv) utility function:

In [2]:
import requests
from ampligraph.datasets import load_from_csv

url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'GoT.csv', sep=',')
X[:5, ]

array([['Smithyton', 'SEAT_OF', 'House Shermer of Smithyton'],
       ['House Mormont of Bear Island', 'LED_BY', 'Maege Mormont'],
       ['Margaery Tyrell', 'SPOUSE', 'Joffrey Baratheon'],
       ['Maron Nymeros Martell', 'ALLIED_WITH',
        'House Nymeros Martell of Sunspear'],
       ['House Gargalen of Salt Shore', 'IN_REGION', 'Dorne']],
      dtype=object)

Let's list the subject and object entities found in the dataset...

In [3]:
entities = np.unique(np.concatenate([X[:, 0], X[:, 2]]))
entities

array(['Abelar Hightower', 'Acorn Hall', 'Addam Frey', ..., 'the Antlers',
       'the Paps', 'unnamed tower'], dtype=object)

... and all of the relationships that link them. Remember, these relationships only link *some* of the entities.

In [4]:
relations = np.unique(X[:, 1])
relations

array(['ALLIED_WITH', 'BRANCH_OF', 'FOUNDED_BY', 'HEIR_TO', 'IN_REGION',
       'LED_BY', 'PARENT_OF', 'SEAT_OF', 'SPOUSE', 'SWORN_TO'],
      dtype=object)


## 2. Defining Train and Test Datasets

As is typical in machine learning, we need to split our dataset into training and test (and sometimes validation) datasets. 

When dealing with Knowledge Graphs, there is a major difference with the standard method of randomly sampling N points to make up our test set. Indeed, each of our data points are two entities linked by some relationship, and we need to ensure that all entities and relationships that are represented in the test set are present in the training set in at least one triple. 

To accomplish this, AmpliGraph provides the [`train_test_split_no_unseen`](https://docs.ampligraph.org/en/latest/generated/ampligraph.evaluation.train_test_split_no_unseen.html#train-test-split-no-unseen) function.  

As an example, we will create a small test size that includes only 100 triples:

In [5]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_test = train_test_split_no_unseen(X, test_size=100) 

Our data is now split into train/test sets. If we needed to further obtain a validation dataset, we can just repeat the same procedure on the test set (and adjusting the split percentages). 

In [6]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (3075, 3)
Test set size:  (100, 3)



## 3. Training a Model

AmpliGraph 2 has a unique class for defining [several](https://docs.ampligraph.org/en/latest/ampligraph.latent_features.html#knowledge-graph-embedding-models) Knoweldge Graph Embedding models (TransE, ComplEx, DistMult, HolE), it sufficies to specify the different scoring type. Together with that, at initialization time, we also need to define some parameters:
- **`k`** : the dimensionality of the embedding space;
- **`eta`** ($\eta$) : the number of negatives (i.e., false triples) that must be generated at training runtime for each positive (i.e., true triple).

To begin with we are going to use the [ComplEx](https://docs.ampligraph.org/en/latest/generated/ampligraph.latent_features.ComplEx.html#ampligraph.latent_features.ComplEx) model.

In [7]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel
model = ScoringBasedEmbeddingModel(k=150,
                                   eta=5,
                                   scoring_type='ComplEx',
                                   seed=0)

Right after defining the model, it is time to compile the model, specifying:

- **`optimizer`** : we will use the Adam optimizer, with a learning rate of 1e-3, but AmpliGraph 2 supports any _tf.keras.optimizers_;
- **`loss`** : we will consider the pairwise loss, with a margin of 0.5 set via the *loss_params* kwarg. However, many other loss functions are supported, and custom losses can be defined by the user;
- **`regularizer`** : we will use the $L_p$ regularization with $p=2$, i.e. L2 regularization. The regularization parameter $\lambda$ = 1e-5 is set via the *regularizer_params* kwarg. Also in this case, _tf.keras.regularizers_ are supported.
- **`initializer`** : we will use the Glorot Uniform initialization, but the _tf.keras.initializers_ are supported.


In [11]:
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=1e-3)
loss = get_loss('pairwise', {'margin': 0.5})
regularizer = get_regularizer('LP', {'p': 2, 'lambda': 1e-5})

model.compile(loss=loss,
              optimizer='adam',
              entity_relation_regularizer=regularizer,
              entity_relation_initializer='glorot_uniform')

AmpliGraph follows the _tensorflow.keras_ style APIs, allowing, after compiling the model, to perform the main operations of the model with the **`fit`**, **`predict`**, and **`evaluate`** methods. 


### Fitting the Model

Once you run the next cell the model will start training. 

On a modern laptop this should take ~3 minutes (although your mileage may vary, especially if you have changed any of the hyper-parameters above).

In [12]:
model.fit(X_train,
          batch_size=5000,
          epochs=200,
          verbose=True)

2023-02-08 23:06:27.469961: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/200


2023-02-08 23:06:28.230759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 668ms/step - loss: 7687.5562
Epoch 2/200
2/2 [==============================] - 0s 43ms/step - loss: 7683.3623
Epoch 3/200
2/2 [==============================] - 0s 48ms/step - loss: 7679.1382
Epoch 4/200
2/2 [==============================] - 0s 40ms/step - loss: 7674.8130
Epoch 5/200
2/2 [==============================] - 0s 48ms/step - loss: 7670.5405
Epoch 6/200
2/2 [==============================] - 0s 44ms/step - loss: 7666.1504
Epoch 7/200
2/2 [==============================] - 0s 41ms/step - loss: 7661.6445
Epoch 8/200
2/2 [==============================] - 0s 41ms/step - loss: 7657.1768
Epoch 9/200
2/2 [==============================] - 0s 46ms/step - loss: 7652.6147
Epoch 10/200
2/2 [==============================] - 0s 43ms/step - loss: 7647.9399
Epoch 11/200
2/2 [==============================] - 0s 41ms/step - loss: 7643.0947
Epoch 12/200
2/2 [==============================] - 0s 41ms/step - loss: 7638.1758
Epoch 13/200
2/2 [======

### Predicting with the Model
After having trained the model, we can use it to predict the score for any unseen triple given that its subject, relation and object were present in the training set.

In [22]:
model.predict(np.array([['Leyton Hightower', 'ALLIED_WITH', 'House Ryswell of the Rills']]))


array([0.09504332], dtype=float32)

### Evaluating
Now it is time to evaluate our model on the test set to see how well it's performing. 

For this we are going to use the `evaluate` method, which takes as arguments:

- **`X_test`** : the data to evaluate on. We're going to use our test set to evaluate.
- **`use_filter`** : whether to filter out the false negatives generated by the corruption strategy. If a dictionary is passed, the values of it are used as elements to filter.
- **`corrupt_side`** : specifies whether to corrupt subj and obj separately or to corrupt both during evaluation.

In [69]:
positives_filter = {'test' : np.concatenate([X_train, X_test])}
ranks = model.evaluate(X_test, 
                       use_filter=positives_filter,   # Corruption strategy filter defined above 
                       corrupt_side='s,o', # corrupt subj and obj separately while evaluating
                       verbose=True)

2023-02-09 15:57:55.492610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 297ms/step


The ***ranks*** returned by the evaluate_performance function indicate the rank at which the test set triple was found when performing link prediction using the model. 

### Metrics
Let's compute some evaluate metrics and print them out.

We're going to use the *mrr_score* (mean reciprocal rank) and *hits_at_n_score* functions. 

- ***mrr_score***:  The function computes the mean of the reciprocal of elements of a vector of rankings ranks.
- ***hits_at_n_score***: The function computes how many elements of a vector of rankings ranks make it to the top n positions.

In [41]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.26
Hits@10: 0.37
Hits@3: 0.28
Hits@1: 0.20


Now, how do we interpret those numbers? 

[Hits@N](http://docs.ampligraph.org/en/1.0.3/generated/ampligraph.evaluation.hits_at_n_score.html#ampligraph.evaluation.hits_at_n_score) indicates how many times in average a true triple was ranked in the top-N. Therefore, on average, we guessed the correct subject or object 53% of the time when considering the top-3 better ranked triples. The choice of which N makes more sense depends on the application.

The [Mean Reciprocal Rank (MRR)](http://docs.ampligraph.org/en/latest/generated/ampligraph.evaluation.mrr_score.html) is another popular metrics to assess the predictive power of a model.


## 4.  Saving and Restoring a Model

Before we go any further, let's save the best model found so that we can restore it in future.

In [43]:
from ampligraph.utils import save_model, restore_model

In [44]:
save_model(model, './best_model.pkl')

WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


This will save the model in the ampligraph_tutorial directory as `best_model.pkl`.

We can then delete the model...

In [45]:
del model

.|.. and then restore it from disk! Ta-da! 

In [46]:
model = restore_model('./best_model.pkl')

Saved model does not include a db file. Skipping.


2023-02-09 13:15:34.968306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-09 13:15:34.990256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


And let's just double check that the model we restored has been fit:

In [47]:
if model.is_fitted:
    print('The model is fit!')
else:
    print('The model is not fit! Did you skip a step?')

The model is fit!



## 5. Predicting New Links

Link prediction allows us to infer missing links in a graph. This has many real-world use cases, such as predicting connections between people in a social network, interactions between proteins in a biological network, and music recommendation based on prior user taste. 

In our case, we're going to see which of the following candidate statements (that we made up) are more likely to be true:


In [48]:
X_unseen = np.array([
    ['Jorah Mormont', 'SPOUSE', 'Daenerys Targaryen'],
    ['Tyrion Lannister', 'SPOUSE', 'Missandei'],
    ["King's Landing", 'SEAT_OF', 'House Lannister of Casterly Rock'],
    ['Sansa Stark', 'SPOUSE', 'Petyr Baelish'],
    ['Daenerys Targaryen', 'SPOUSE', 'Jon Snow'],
    ['Daenerys Targaryen', 'SPOUSE', 'Craster'],
    ['House Stark of Winterfell', 'IN_REGION', 'The North'],
    ['House Stark of Winterfell', 'IN_REGION', 'Dorne'],
    ['House Tyrell of Highgarden', 'IN_REGION', 'Beyond the Wall'],
    ['Brandon Stark', 'ALLIED_WITH', 'House Stark of Winterfell'],
    ['Brandon Stark', 'ALLIED_WITH', 'House Lannister of Casterly Rock'],    
    ['Rhaegar Targaryen', 'PARENT_OF', 'Jon Snow'],
    ['House Hutcheson', 'SWORN_TO', 'House Tyrell of Highgarden'],
    ['Daenerys Targaryen', 'ALLIED_WITH', 'House Stark of Winterfell'],
    ['Daenerys Targaryen', 'ALLIED_WITH', 'House Lannister of Casterly Rock'],
    ['Jaime Lannister', 'PARENT_OF', 'Myrcella Baratheon'],
    ['Robert I Baratheon', 'PARENT_OF', 'Myrcella Baratheon'],
    ['Cersei Lannister', 'PARENT_OF', 'Myrcella Baratheon'],
    ['Cersei Lannister', 'PARENT_OF', 'Brandon Stark'],
    ["Tywin Lannister", 'PARENT_OF', 'Jaime Lannister'],
    ["Missandei", 'SPOUSE', 'Grey Worm'],
    ["Brienne of Tarth", 'SPOUSE', 'Jaime Lannister']
])

In [51]:
positives_filter['test'] = np.vstack((positives_filter['test'], X_unseen))

In [55]:
ranks_unseen = model.evaluate(X_unseen,
                              use_filter=positives_filter,   # Corruption strategy filter defined above 
                              corrupt_side = 's+o',
                              verbose=True)

2023-02-09 13:19:35.033726: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-09 13:19:35.252684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 401ms/step


In [56]:
scores = model.predict(X_unseen)

2023-02-09 13:19:40.424379: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


We transform the scores (real numbers) into probabilities (bound between 0 and 1) using the expit transform.

Note that the probabilities are not calibrated in any sense. 

Advanced note: To calibrate the probabilities, one may use a procedure such as [Platt scaling](https://en.wikipedia.org/wiki/Platt_scaling) or [Isotonic regression](https://en.wikipedia.org/wiki/Isotonic_regression). The challenge is to define what is a true triple and what is a false one, as the calibration of the probability of a triple being true depends on the base rate of positives and negatives.

In [57]:
from scipy.special import expit
probs = expit(scores)

In [60]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score", ascending=False)

,statement,rank,score,prob
12,House Hutcheson SWORN_TO House Tyrell of Highg...,[-1],0.830148,0.696386
9,Brandon Stark ALLIED_WITH House Stark of Winte...,[-1],0.630217,0.652539
6,House Stark of Winterfell IN_REGION The North,[7],0.235329,0.558562
8,House Tyrell of Highgarden IN_REGION Beyond th...,[206],0.110351,0.527560
3,Sansa Stark SPOUSE Petyr Baelish,[499],0.084068,0.521005
7,House Stark of Winterfell IN_REGION Dorne,[637],0.079129,0.519772
13,Daenerys Targaryen ALLIED_WITH House Stark of ...,[688],0.058003,0.514497
15,Jaime Lannister PARENT_OF Myrcella Baratheon,[674],0.055809,0.513949
20,Missandei SPOUSE Grey Worm,[650],0.046343,0.511584
2,King's Landing SEAT_OF House Lannister of Cast...,[1066],0.032168,0.508041


We see that the embeddings captured some truths about Westeros. For example, **House Stark is placed in the North rather than Dorne**. It also realises **Daenerys Targaryen has no relation with Craster**, **nor Tyrion with Missandei**. It captures random trivia, as **House Hutcheson is indeed in the Reach and sworn to the Tyrells**. On the other hand, some marriages that it predicts never really happened. These mistakes are understandable: those characters were indeed close and appeared together in many different circumstances. 


## 6. Visualizing Embeddings with Tensorboard projector

The kind folks at Google have created [Tensorboard](https://www.tensorflow.org/tensorboard), which allows us to graph how our model is learning (or not :|), peer into the innards of neural networks, and [visualize high-dimensional embeddings in the browser](https://projector.tensorflow.org/).   

Lets import the [`create_tensorboard_visualization`](http://docs.ampligraph.org/en/1.0.3/generated/ampligraph.utils.create_tensorboard_visualizations.html#ampligraph.utils.create_tensorboard_visualizations) function, which simplifies the creation of the files necessary for Tensorboard to display the embeddings.

In [61]:
from ampligraph.utils import create_tensorboard_visualizations

And now we'll run the function with our model, specifying the output path:

In [62]:
create_tensorboard_visualizations(model, 'GoT_embeddings')

If all went well, we should now have a number of files in the `AmpliGraph/tutorials/GoT_embeddings` directory:

```
GoT_embeddings/
    ├── checkpoint
    ├── embeddings_projector.tsv
    ├── graph_embedding.ckpt.data-00000-of-00001
    ├── graph_embedding.ckpt.index
    ├── graph_embedding.ckpt.meta
    ├── metadata.tsv
    └── projector_config.pbtxt
```

To visualize the embeddings in Tensorboard, run the following from your command line inside `AmpliGraph/tutorials`:

```bash
tensorboard --logdir=./visualizations
```
    
.. and once your browser opens up you should be able to see and explore your embeddings as below (PCA-reduced, two components):

![](img/GoT_tensoboard.png)





# The End

You made it to the end! Well done!

For more information please visit the [AmpliGraph GitHub](github.com/Accenture/AmpliGraph) (and remember to star the project!), or check out the [documentation](docs.ampligraph.org).